In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pdfx
from PyPDF2 import PdfFileReader
from urllib.request import urlretrieve
from difflib import SequenceMatcher


In [ ]:
table = pd.read_excel('InPharmD_ML_DATASET.xlsx')

In [ ]:
table = table[:20]

**Scrape Titles From PDF**

In [ ]:
def get_title(path):
    with open(path, 'rb') as f:
        pdf = PdfFileReader(f)
        info = pdf.getDocumentInfo()
    return info.title

def download_file(download_url, fileNo):
    urlretrieve(download_url, '/Data/file' + str(fileNo))

1. Get PDF source links:

In [ ]:
download_links = []

In [ ]:
index = 0;
while index < len(table):
  print(index)
  url = requests.get(table['PDF'][index]).text
  
  try:
    soup = BeautifulSoup(url, 'html.parser')
    src = soup.find('iframe')['src']
    if src.startswith('//'):
      download_links.append('https:' + src)
      print("added http: to " + download_links[index])
    else:
      download_links.append(src)
      print("normal sci-hub: " + download_links[index])
    index = index + 1
    continue  

  except:
    if 'pdf' in table['PDF'][index]:
      download_links.append(table['PDF'][index])
      print("non-scihub PDF: " + download_links[index])
      index = index + 1
      continue
    download_links.append("Must check manually!")
    print(download_links[index])
    index = index + 1
    continue

In [ ]:
table['DownloadLinks'] = download_links

2. Download files and retrieve title:

In [ ]:
i = 0
titles = []
for link in table['Download Links']:
  
  try:
    download_file(link, i)
    file = ('/Data/file' + str(i))
    print(file)
    titles.append(get_title(file))
    !rm {file}
    
  except:
    titles.append('Check Manually')
  
  print(str(i) + ": " + str(titles[i])) 
  i = i + 1

In [ ]:
j = 0
success = 0;
for t in titles:
  if SequenceMatcher(None, str(t), table['Citation'][j]).ratio() >= .3:
    success = success + 1
print(success/len(table))

0.012345679012345678


In [ ]:

errors = []
k = 0
for link in table['Download Links']:
  print(k)
  if link == ("Must check manually!" or "None" or "untitled" or ""):
    errors.append(k)
  k = k + 1
errors

Fix errors in excel document manually